In [8]:
import os
import csv
import time
import gensim
from gensim.models.callbacks import CallbackAny2Vec

csv.field_size_limit(1000000)

131072

In [ ]:
# create iterator for corpus training
import csv

class PlaylistCorpus(object):
    """An interator that yields sentences (lists of str)."""
    spamreader = csv.reader(open('playlists.csv', 'r', encoding='utf-8'),delimiter=';')

    def __iter__(self):
        for row in self.spamreader:
            columns = str(row)[2:-2].split(';')
            # if len(columns) < 1000: #limit length of playlists?
            #explore gensim preprocessing
            playlist = []
            for column in columns[2:]:
                playlist.append(column)
            yield playlist

In [ ]:
playlists = PlaylistCorpus()

In [ ]:
for i, playlist in enumerate(playlists):
    if i > 1:
        break
    print(i)
    print (playlist)

In [9]:
from gensim.models.callbacks import CallbackAny2Vec

# hyperparameters

min_count = 10      # minimum number of occurences of a track in all playlists
window = 2          # length of sequences
embedding_dim = 100 # number of dimensions in hidden layer
batch_words = 10000 # number of tracks to process in each batch
iter = 10           # number of iterations
sg = 1              # skip-gram (1) or Continuous Bag Of Words (0)

# ------------------------------------------------------------------------------

valid_examples = [
    '2NMgVh5qaPprKTEzFe3501', # The Police - Roxanne
    '3Ti0GdlrotgwsAVBBugv0I', # A Tribe Called Quest - Can I Kick It?
    '0nyrltZrQGAJMBZc1bYvuQ', # James Brown - Get Up Offa That Thing
    '4hy4fb5D1KL50b3sng9cjw', # Nirvana - Smells Like Teen Spirit
    '1P49MJhU5vzttesFxw3dOM', # Bob Marley & The Wailers - Three Little Birds
    '76GlO5H5RT6g7y0gev86Nk', # The Cure - Just Like Heaven
    '40tAOP3DPqmVD6L1h45Jp6', # Frank Sinatra - My Way
    '4IMvgp0WZqr9mRqpEvDKxI', # The Clash - Rock the Casbah
    '1iDcKYNvo6gglrOG6lvnHL', # The Rolling Stones - Sympathy For The Devil
    '5uvosCdMlFdTXhoazkTI5R', # The Doors - Light My Fire
    '15JINEqzVMv3SvJTAXAKED', # Eminem - Love The Way You Lie
    '69kOkLUCkxIZYexIgSG8rq', # Daft Punk - Get Lucky
    '6oVY50pmdXqLNVeK8bzomn', # John Coltrane - My Favorite Things
    '6ui6l3ZNvlrGQZArwo8195', # Sex Pistols - God Save The Queen
    '0YammaEkYSeo9vQYZ1OwS6', # David Guetta - Say My Name
    '4SHZsQIdS2N1E5yqvoXF8o'  # Andy Williams - Can't Take My Eyes Off You
]

# ------------------------------------------------------------------------------

class logger(CallbackAny2Vec):
    def __init__(self):
        print('Starting...')
        self.epoch = 0
        self.loss = 0

    def on_train_begin(self, model):
        self.start = time.time()
        print(f'Starting training......')

    def on_epoch_end(self, model):
        elapsed = time.time() - self.start
        print('#{}'.format(self.epoch), 'loss =',
              (model.get_latest_training_loss() - self.loss) / batch_words,
              'elapsed time =', elapsed // 60, 'minutes', elapsed % 60, 'seconds')
        self.epoch += 1
        self.loss = model.get_latest_training_loss()
        print('Saving model...')
        model.save('word2vec.model')
        # _model = gensim.models.Word2Vec.load('word2vec.model')
        # for track in valid_examples:
        #     similar = _model.wv.most_similar(positive=[track], topn=8)
        #     most_similar = ''
        #     for i in range(0, 8):
        #         most_similar = most_similar + '%s (%.2f)' % (tracks[similar[i][0]][0], similar[i][1]) + ', '
        #     print('  %s -> %s' % (tracks[track][0], most_similar))
        # print()
        # del _model

In [60]:
model = gensim.models.Word2Vec(sentences=playlists, size=embedding_dim,
                               min_count=min_count, window=window, iter=iter,
                               batch_words=batch_words, compute_loss=True, sg=True,
                               callbacks=[logger()],workers=4)
print(model)

Starting...
Starting training......
#0 loss = 8123.3512 elapsed time = 19.0 minutes 46.67325305938721 seconds
Saving model...
#1 loss = 2663.0968 elapsed time = 39.0 minutes 3.927161931991577 seconds
Saving model...
#2 loss = 2610.2576 elapsed time = 58.0 minutes 25.13147783279419 seconds
Saving model...
#3 loss = 25.0672 elapsed time = 76.0 minutes 50.87499690055847 seconds
Saving model...
#4 loss = 0.0 elapsed time = 94.0 minutes 47.31240892410278 seconds
Saving model...
#5 loss = 0.0 elapsed time = 113.0 minutes 40.097012996673584 seconds
Saving model...
#6 loss = 0.0 elapsed time = 132.0 minutes 56.45785689353943 seconds
Saving model...
#7 loss = 0.0 elapsed time = 152.0 minutes 10.723788022994995 seconds
Saving model...
#8 loss = 0.0 elapsed time = 171.0 minutes 36.7718939781189 seconds
Saving model...
#9 loss = 0.0 elapsed time = 190.0 minutes 14.955538749694824 seconds
Saving model...
Word2Vec(vocab=2924687, size=100, alpha=0.025)


In [62]:
model.save('song2vec.model')

In [63]:
# model.wv.save_word2vec_format(fname='song2vec.bin',fvocab='songVocab.bin',binary=True)

In [10]:
import gensim

model = gensim.models.Word2Vec.load('song2vec.model')
print(model)

Word2Vec(vocab=2924687, size=100, alpha=0.025)


In [11]:
print('Read in tracks...')
tracks = {}
with open('tracks.csv', "r", encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        columns = str(row)[2:-2].split(';')
        tracks[columns[0]] = [columns[1] + ' - ' +
                              columns[2], columns[3]] # title - artist, url
print(f'Number of tracks: {len(tracks)}')

Read in tracks...
Number of tracks: 19817445


In [13]:
for track in valid_examples:
            similar = model.wv.most_similar(positive=[track], topn=20)
            most_similar = ''
            for i in range(0, 20):
                most_similar = most_similar + '%s (%.2f)' % (tracks[similar[i][0]][0], similar[i][1]) + ', '
            print('  %s -> %s' % (tracks[track][0], most_similar))

The Police - Roxanne -> The Police - Can't Stand Losing You (0.83), The Police - Don't Stand So Close To Me (0.83), The Police - So Lonely (0.82), The Police - Message In A Bottle (0.82), Peter Gabriel - Sledgehammer - Live (0.81), U2 - Sunday Bloody Sunday (0.81), The Police - Roxanne (0.81), The Police - Every Little Thing She Does Is Magic (0.81), The Police - Walking On The Moon (0.80), Midnight Oil - Beds Are Burning (0.80), Queen - I Want To Break Free - Remastered 2011 (0.80), The Rolling Stones - Start Me Up - 2009 Re-Mastered Digital Version (0.80), Sting - Englishman In New York (0.80), The Police - Message In A Bottle (0.80), The Police - Every Breath You Take (0.80), Queen - Crazy Little Thing Called Love (0.80), Europe - The Final Countdown (0.79), U2 - Pride (In The Name Of Love) (0.79), Roy Orbison - Oh, Pretty Woman (0.79), The Police - Every Little Thing She Does Is Magic (0.79), 
  A Tribe Called Quest - Can I Kick It? -> A Tribe Called Quest - Electric Relaxation (0.